# tensorflow2.0 基础
本章为tensorflow 2.0基本操作
## 张量计算
tensorflow2.0的新特性之一就是可以不用创建流图直接计算。
在 TensorFlow 1.X 版本中，**必须**在导入**TensorFlow**库后调用`tf.enable_eager_execution()`函数以启用 Eager Execution 模式。在 TensorFlow 2.0 版本中，Eager Execution 模式将成为默认模式，无需额外调用`tf.enable_eager_execution()`函数（不过若要关闭 Eager Execution，则需调用`tf.compat.v1.disable_eager_execution()`函数）。

In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [19]:
A = tf.constant([[1., 2.], [3., 4.]])
B = tf.constant([[5., 6.], [7., 8.]])
C = tf.matmul(A, B)
print(C)

tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)


可以看出，A和B作为一个矩阵，在运行矩阵乘法之后输出的C直接就是计算结果，而同样的语句在tensorflow-1.8.0中运行则为：
> Tensor("MatMul:0", shape=(2, 2), dtype=int32)

相同的语句在tensorflow1.0+版本中只是进行了运算流图的创建，将矩阵相乘作为第1个节点，如果要计算并获得结果，则必须通过`tf.Session()`中的`run()`方法进行流图计算。
```python
with tf.Session() as sess
    sess.run(C)
    print(ans)
    
>>>[[19 22]
 [43 50]]
```
这点是tensorflow-2.0版本更迭中最大的改动，通过动态图的方式替换原先的静态图，构图的同时进行tensor运算，极大地减少了复杂度，提高了可观性。

## 自动求导机制
在机器学习中，经常需要计算函数的导数。tensorflow提供了自动求导机制，在`tf.GradientTape()`下计算$y=x^2 当x=3$时的导数

In [20]:
x = tf.Variable(initial_value=tf.constant(3.),name='x',validate_shape=[1])
with tf.GradientTape() as tape:
    y = tf.square(x) # 在tf.GradientTape()的上下文内，所有计算步骤都会被记录以用于求导
dy_dx = tape.gradient(y, x) # 计算y关于x的导数
print([y.numpy(), dy_dx.numpy()])

[9.0, 6.0]


这里`x`是一个初始化为3的变量（Variable），通过`tf.Variable`直接进行声明，并指定了初始值为float类型的`3.`。变量的意义在于，它默认能够被tensorflow的自动求导机制所求导，因此常用以定义模型参数。`tf.GradientTape()`是一个自动求导的记录器，在其中的变量和计算步骤都会被自动记录。上面的示例中，变量`x`和计算步骤`y = tf.square(x)`被自动记录，因此可以通过`y_grad = tape.gradient(y, x)`求张量`y`对变量`x`的导数。

更加常见的，是对多元函数求偏导。以下展示计算函数$L(w, b) = \|Xw + b - y\|^2$在$w = (1, 2)^T, b = 1$时分别对$w, b$的偏导数。其中$X = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}, y = \begin{bmatrix} 1 \\ 2\end{bmatrix}$。

In [26]:
X = tf.constant([[1., 2.], [3., 4.]])
Y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]], name='w')
b = tf.Variable(initial_value=1., name='b')
with tf.GradientTape() as tape:
    L = 0.5 * tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y)) 
w_grad, b_grad = tape.gradient(L, [w, b]) # 计算L(w,b)关于w，b的偏导数
print([L.numpy(), w_grad.numpy(), b_grad.numpy()])

[62.5, array([[35.],
       [50.]], dtype=float32), 15.0]


这里，`tf.square()`操作代表对输入张量中每个元素求平方（不改变形状）。`tf.reduce_sum()`表示对输入张量的所有元素求和。从输出可见，TensorFlow 帮助我们计算出了
$$L((1, 2)^T, 1) = 62.5$$
$$\frac{\partial L(w, b)}{\partial w} |_{w = (1, 2)^T, b = 1} = \begin{bmatrix} 35 \\ 50\end{bmatrix}$$ 
$$\frac{\partial L(w, b)}{\partial b} |_{w = (1, 2)^T, b = 1} = 15$$

## Tensorflow下的线性回归
TensorFlow 的 Eager Execution（动态图）模式提供快速运算、自动求导、优化器等一系列对深度学习非常重要的功能。以下展示如何使用tensorflow计算线性回归。

In [28]:
import numpy as np

X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
for e in range(num_epoch):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y))
    # tensorflow自动计算损失函数关于自变量（模型参数）的梯度
    grads = tape.gradient(loss, variables)
    # tensorflow自动根据梯度更新参数
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))
    
print(a, b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.97637> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.057565063>


我们使用了计算了损失函数关于参数的偏导数。同时，使用`tf.keras.optimizers.SGD(learning_rate=1e-3)`声明了一个梯度下降**优化器**（Optimizer），其学习率为`1e-3`。优化器可以帮助我们根据计算出的求导结果更新模型参数，从而最小化某个特定的损失函数，具体使用方式是调用其`apply_gradients()`方法。   

注意到这里，更新模型参数的方法`optimizer.apply_gradients()`需要提供参数`grads_and_vars`，即待更新的变量（如上述代码中的 `variables` ）及损失函数关于这些变量的偏导数（如上述代码中的`grads`）。具体而言，这里需要传入一个 Python 列表（List），列表中的每个元素是一个 （变量的偏导数，变量） 对。比如这里是`[(grad_a, a), (grad_b, b)]`。我们通过`grads = tape.gradient(loss, variables)`求出 tape 中记录的`loss`关于`variables = [a, b]`中每个变量的偏导数，也就是`grads = [grad_a, grad_b]`，再使用 Python 的`zip()`函数将`grads = [grad_a, grad_b]`和`variables = [a, b]`拼装在一起，就可以组合出所需的参数了。